# Imports & Load Data
作業に必要なライブラリをインポートして、 以下のデータを読み込みます。

* stock_price : 株価情報
* stock_list : 銘柄情報
* stock_fin : 財務諸表
* stock_labels : 目的変数

In [1]:
!pip install catboost==0.24.4 

    100% |████████████████████████████████| 65.7MB 140kB/s eta 0:00:01 2% |█                               | 2.0MB 4.9MB/s eta 0:00:14    29% |█████████▌                      | 19.6MB 6.6MB/s eta 0:00:08    58% |██████████████████▋             | 38.2MB 11.9MB/s eta 0:00:03    72% |███████████████████████▏        | 47.5MB 12.9MB/s eta 0:00:02
    100% |████████████████████████████████| 13.2MB 3.0MB/s eta 0:00:01    89% |████████████████████████████▊   | 11.8MB 12.7MB/s eta 0:00:01
  Stored in directory: /root/.cache/pip/wheels/d7/a9/33/acc7b709e2a35caa7d4cae442f6fe6fbf2c43f80823d46460c
Successfully built retrying


In [2]:
import os
import pickle
import sys
import warnings
from glob import glob

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import shap
import  xgboost as xgb
from scipy.stats import spearmanr
from sklearn.ensemble import (
    ExtraTreesRegressor,
    GradientBoostingRegressor,
    RandomForestRegressor,
)
from sklearn.metrics import accuracy_score, mean_squared_error
from tqdm.auto import tqdm
from catboost import CatBoostRegressor


# 表示用の設定を変更します
%matplotlib inline
pd.options.display.max_rows = 100
pd.options.display.max_columns = 100
pd.options.display.width = 120

calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.


In [3]:
# python 3.7.3であることを確認します
import sys
print(sys.version)

3.7.3 (default, Mar 27 2019, 22:11:17) 
[GCC 7.3.0]


In [4]:
# データセット保存先ディレクトリ（""の中身はご自身の環境に合わせて定義してください。）
dataset_dir="/path/to"

In [5]:
# 読み込むファイルを定義します。
inputs = {
    "stock_list": f"{dataset_dir}/stock_list.csv.gz",
    "stock_price": f"{dataset_dir}/stock_price.csv.gz",
    "stock_fin": f"{dataset_dir}/stock_fin.csv.gz",
    # 本チュートリアルでは使用しないため、コメントアウトしています。
    # "stock_fin_price": f"{dataset_dir}/stock_fin_price.csv.gz",
    "stock_labels": f"{dataset_dir}/stock_labels.csv.gz",
}

# ファイルを読み込みます
dfs = {}
for k, v in inputs.items():
    print(k)
    dfs[k] = pd.read_csv(v)

stock_list
stock_price
stock_fin
stock_labels


# 特徴量の生成

In [6]:
FEATURES = ['MA_gap_2month',
            'MA_gap_3month',
            'volatility_2month',
            'volatility_3month',
            'Result_Dividend FiscalYear',
            'return_3month',
            'Forecast_Dividend FiscalYear',
            'volatility_1month',
            'Forecast_FinancialStatement FiscalYear',
            'MA_gap_1month',
            'pbr',
            'Result_FinancialStatement FiscalYear',
            'return_1month',
            'ema_12',
            'Result_FinancialStatement TotalAssets',
            'signal',
            'Previous_FinancialStatement NetIncome',
            'per',
            'Result_FinancialStatement CashFlowsFromOperatingActivities',
            'Result_FinancialStatement CashFlowsFromInvestingActivities',
            'ema_10']

FEATURES_HIGH = ['MA_gap_2month_high',
                 'MA_gap_3month_high',
                 'volatility_2month_high',
                 'volatility_3month_high',
                 'Result_Dividend FiscalYear',
                 'return_3month_high',
                 'Forecast_Dividend FiscalYear',
                 'volatility_1month_high',
                 'Forecast_FinancialStatement FiscalYear',
                 'MA_gap_1month_high',
                 'pbr',
                 'Result_FinancialStatement FiscalYear',
                 'return_1month_high',
                 'ema_12',
                 'Result_FinancialStatement TotalAssets',
                 'signal',
                 'Previous_FinancialStatement NetIncome',
                 'per',
                 'Result_FinancialStatement CashFlowsFromOperatingActivities',
                 'Result_FinancialStatement CashFlowsFromInvestingActivities',
                 'ema_10']

FEATURES_LOW = ['MA_gap_2month_low',
                'MA_gap_3month_low',
                'volatility_2month_low',
                'volatility_3month_low',
                'Result_Dividend FiscalYear',
                'return_3month_low',
                'Forecast_Dividend FiscalYear',
                'volatility_1month_low',
                'Forecast_FinancialStatement FiscalYear',
                'MA_gap_1month_low',
                'pbr',
                'Result_FinancialStatement FiscalYear',
                'return_1month_low',
                'ema_12',
                'Result_FinancialStatement TotalAssets',
                'signal',
                'Previous_FinancialStatement NetIncome',
                'per',
                'Result_FinancialStatement CashFlowsFromOperatingActivities',
                'Result_FinancialStatement CashFlowsFromInvestingActivities',
                'ema_10']


In [7]:
def get_feature_columns(dfs, train_X, column_group="fundamental+technical"):
    # 特徴量グループを定義
    # ファンダメンタル
    fundamental_cols = dfs["stock_fin"].select_dtypes("float64").columns
    fundamental_cols = fundamental_cols[
        fundamental_cols != "Result_Dividend DividendPayableDate"
        ]
    fundamental_cols = fundamental_cols[fundamental_cols != "Local Code"]
    # 価格変化率
    returns_cols = [x for x in train_X.columns if "return" in x]
    # テクニカル
    technical_cols = [
        x for x in train_X.columns if
        (x not in fundamental_cols) and (x != "code")
    ]
    columns = {
        "fundamental_only": fundamental_cols,
        "return_only": returns_cols,
        "technical_only": technical_cols,
        "fundamental+technical": list(fundamental_cols) + list(
            technical_cols),
        "selected_columns":FEATURES,
        "selected_high_columns":FEATURES_HIGH,
        "selected_low_columns":FEATURES_LOW,
    }
    return columns[column_group]

In [8]:
model_path = os.path.join(os.path.dirname("__file__"), "../../model")
test_X_path = os.path.join(os.path.dirname("__file__"), "../../../high_low_datas/test_X")
test_y_path = os.path.join(os.path.dirname("__file__"), "../../../high_low_datas/test_y")
val_X_path = os.path.join(os.path.dirname("__file__"), "../../../high_low_datas/val_X")
val_y_path = os.path.join(os.path.dirname("__file__"), "../../../high_low_datas/val_y")
train_X_path = os.path.join(os.path.dirname("__file__"), "../../../high_low_datas/train_X")
train_y_path = os.path.join(os.path.dirname("__file__"), "../../../high_low_datas/train_y")

In [9]:
# 対象の目的変数を定義
labels = {
#    "label_high_5",
#    "label_high_10",
    "label_high_20",
#    "label_low_5",
#    "label_low_10",
    "label_low_20",
}

In [10]:
# 特徴量追加済みデータ
proceed_datas = {
    "train_X",
    "train_y",
    "val_X",
    "val_y",
    "test_X",
    "test_y"
}

In [11]:
# ライブラリインポート
from sklearn.model_selection import GridSearchCV


# reg_cv = GridSearchCV(CatBoostRegressor(), {
#     "eta": [0.01, 0.05, 0.1], 
#     "gamma": [0.1,0.2,0.3,0.4,0.5],
#     "n_estimators": [50, 100, 200], 
#     "max_depth": [5, 7, 9,10],
#     "subsample":[0.6,0.8,1],
#     "colsample_bytree": [0.5,0.7,0.9],
#     "random_seed": [0],
# }, verbose=1)

reg_cv = GridSearchCV(CatBoostRegressor(), 
                      {'iterations': [100, 222], 
                       'depth': [5, 7, 9], 
                       'learning_rate': [0.01,0.18831273426065617, 0.2],               
                       'random_strength': [33], 
                       'bagging_temperature': [0.01], 
                       'od_type': ['Iter'], 
                       'od_wait': [5, 10, 21],
    }, verbose=1)


In [12]:
label = "label_high_20"

In [14]:
data_X = os.path.join(train_X_path, f"train_X_{label}.pkl")
with open(data_X , "rb") as f:
    train_X = pickle.load(f)
data_y = os.path.join(train_y_path, f"train_y_{label}.pkl")
with open(data_y , "rb") as f:
    train_y = pickle.load(f)

feature_columns = get_feature_columns(dfs, train_X, column_group='selected_high_columns')
# 訓練実施
reg_cv.fit(train_X[feature_columns].values, train_y.values)

You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 54 candidates, totalling 162 fits
0:	learn: 0.1609284	total: 58.9ms	remaining: 5.83s
1:	learn: 0.1608981	total: 64.6ms	remaining: 3.17s
2:	learn: 0.1608658	total: 72.1ms	remaining: 2.33s
3:	learn: 0.1608377	total: 81.7ms	remaining: 1.96s
4:	learn: 0.1608270	total: 90.8ms	remaining: 1.72s
5:	learn: 0.1607593	total: 98.8ms	remaining: 1.55s
6:	learn: 0.1607005	total: 106ms	remaining: 1.41s
7:	learn: 0.1606648	total: 113ms	remaining: 1.3s
8:	learn: 0.1606186	total: 120ms	remaining: 1.21s
9:	learn: 0.1605741	total: 128ms	remaining: 1.15s
10:	learn: 0.1605008	total: 136ms	remaining: 1.1s
11:	learn: 0.1604789	total: 149ms	remaining: 1.09s
12:	learn: 0.1604241	total: 158ms	remaining: 1.06s
13:	learn: 0.1603609	total: 166ms	remaining: 1.02s
14:	learn: 0.1603340	total: 173ms	remaining: 978ms
15:	learn: 0.1602679	total: 180ms	remaining: 947ms
16:	learn: 0.1602006	total: 188ms	remaining: 916ms
17:	learn: 0.1601678	total: 193ms	remaining: 881ms
18:	learn: 0.1601259	total

82:	learn: 0.1808449	total: 390ms	remaining: 79.9ms
83:	learn: 0.1808122	total: 395ms	remaining: 75.3ms
84:	learn: 0.1807913	total: 400ms	remaining: 70.5ms
85:	learn: 0.1807646	total: 405ms	remaining: 65.9ms
86:	learn: 0.1807395	total: 410ms	remaining: 61.3ms
87:	learn: 0.1806862	total: 415ms	remaining: 56.6ms
88:	learn: 0.1806678	total: 419ms	remaining: 51.8ms
89:	learn: 0.1806252	total: 423ms	remaining: 47ms
90:	learn: 0.1805965	total: 427ms	remaining: 42.2ms
91:	learn: 0.1805719	total: 431ms	remaining: 37.5ms
92:	learn: 0.1805457	total: 435ms	remaining: 32.7ms
93:	learn: 0.1805274	total: 438ms	remaining: 28ms
94:	learn: 0.1805266	total: 440ms	remaining: 23.2ms
95:	learn: 0.1804808	total: 445ms	remaining: 18.5ms
96:	learn: 0.1804563	total: 448ms	remaining: 13.9ms
97:	learn: 0.1804002	total: 452ms	remaining: 9.22ms
98:	learn: 0.1803786	total: 455ms	remaining: 4.59ms
99:	learn: 0.1803437	total: 459ms	remaining: 0us
0:	learn: 0.1953640	total: 3.57ms	remaining: 353ms
1:	learn: 0.1953259	

55:	learn: 0.1586755	total: 240ms	remaining: 189ms
56:	learn: 0.1586445	total: 246ms	remaining: 185ms
57:	learn: 0.1586197	total: 251ms	remaining: 181ms
58:	learn: 0.1586003	total: 257ms	remaining: 178ms
59:	learn: 0.1585756	total: 260ms	remaining: 173ms
60:	learn: 0.1585539	total: 265ms	remaining: 169ms
61:	learn: 0.1585251	total: 268ms	remaining: 164ms
62:	learn: 0.1585044	total: 273ms	remaining: 160ms
63:	learn: 0.1584724	total: 278ms	remaining: 156ms
64:	learn: 0.1584603	total: 283ms	remaining: 153ms
65:	learn: 0.1584317	total: 286ms	remaining: 147ms
66:	learn: 0.1583952	total: 290ms	remaining: 143ms
67:	learn: 0.1583750	total: 295ms	remaining: 139ms
68:	learn: 0.1583477	total: 300ms	remaining: 135ms
69:	learn: 0.1583106	total: 305ms	remaining: 131ms
70:	learn: 0.1582840	total: 310ms	remaining: 126ms
71:	learn: 0.1582591	total: 313ms	remaining: 122ms
72:	learn: 0.1582184	total: 317ms	remaining: 117ms
73:	learn: 0.1581916	total: 322ms	remaining: 113ms
74:	learn: 0.1581633	total: 326

26:	learn: 0.1937579	total: 134ms	remaining: 362ms
27:	learn: 0.1936807	total: 139ms	remaining: 357ms
28:	learn: 0.1936262	total: 144ms	remaining: 352ms
29:	learn: 0.1935612	total: 150ms	remaining: 349ms
30:	learn: 0.1935221	total: 154ms	remaining: 343ms
31:	learn: 0.1934579	total: 158ms	remaining: 335ms
32:	learn: 0.1933720	total: 161ms	remaining: 327ms
33:	learn: 0.1932991	total: 166ms	remaining: 322ms
34:	learn: 0.1932167	total: 170ms	remaining: 317ms
35:	learn: 0.1931393	total: 174ms	remaining: 310ms
36:	learn: 0.1930960	total: 179ms	remaining: 305ms
37:	learn: 0.1930778	total: 183ms	remaining: 299ms
38:	learn: 0.1930058	total: 188ms	remaining: 293ms
39:	learn: 0.1929594	total: 190ms	remaining: 286ms
40:	learn: 0.1929215	total: 194ms	remaining: 279ms
41:	learn: 0.1928952	total: 198ms	remaining: 273ms
42:	learn: 0.1928504	total: 202ms	remaining: 267ms
43:	learn: 0.1928082	total: 205ms	remaining: 261ms
44:	learn: 0.1927468	total: 209ms	remaining: 255ms
45:	learn: 0.1926825	total: 211

...

175:	learn: 0.1133899	total: 1.98s	remaining: 517ms
176:	learn: 0.1131340	total: 2s	remaining: 507ms
177:	learn: 0.1129198	total: 2.01s	remaining: 497ms
178:	learn: 0.1127988	total: 2.02s	remaining: 486ms
179:	learn: 0.1125202	total: 2.04s	remaining: 475ms
180:	learn: 0.1123437	total: 2.05s	remaining: 465ms
181:	learn: 0.1120576	total: 2.06s	remaining: 454ms
182:	learn: 0.1115791	total: 2.08s	remaining: 443ms
183:	learn: 0.1113315	total: 2.09s	remaining: 432ms
184:	learn: 0.1109910	total: 2.1s	remaining: 421ms
185:	learn: 0.1108322	total: 2.12s	remaining: 410ms
186:	learn: 0.1105505	total: 2.13s	remaining: 399ms
187:	learn: 0.1102894	total: 2.14s	remaining: 388ms
188:	learn: 0.1100733	total: 2.16s	remaining: 376ms
189:	learn: 0.1098835	total: 2.17s	remaining: 365ms
190:	learn: 0.1096815	total: 2.18s	remaining: 353ms
191:	learn: 0.1094521	total: 2.19s	remaining: 342ms
192:	learn: 0.1093268	total: 2.2s	remaining: 330ms
193:	learn: 0.1091463	total: 2.21s	remaining: 319ms
194:	learn: 0.109

124:	learn: 0.1360237	total: 1.46s	remaining: 1.13s
125:	learn: 0.1355897	total: 1.47s	remaining: 1.12s
126:	learn: 0.1352900	total: 1.49s	remaining: 1.11s
127:	learn: 0.1349833	total: 1.5s	remaining: 1.1s
128:	learn: 0.1345607	total: 1.51s	remaining: 1.09s
129:	learn: 0.1343074	total: 1.52s	remaining: 1.07s
130:	learn: 0.1338903	total: 1.53s	remaining: 1.06s
131:	learn: 0.1334566	total: 1.54s	remaining: 1.05s
132:	learn: 0.1330963	total: 1.55s	remaining: 1.04s
133:	learn: 0.1328324	total: 1.56s	remaining: 1.03s
134:	learn: 0.1324775	total: 1.57s	remaining: 1.01s
135:	learn: 0.1321873	total: 1.59s	remaining: 1s
136:	learn: 0.1318128	total: 1.6s	remaining: 993ms
137:	learn: 0.1314884	total: 1.63s	remaining: 992ms
138:	learn: 0.1313746	total: 1.65s	remaining: 984ms
139:	learn: 0.1312438	total: 1.67s	remaining: 980ms
140:	learn: 0.1310145	total: 1.7s	remaining: 975ms
141:	learn: 0.1306060	total: 1.71s	remaining: 965ms
142:	learn: 0.1301308	total: 1.73s	remaining: 955ms
143:	learn: 0.12977

[Parallel(n_jobs=1)]: Done 162 out of 162 | elapsed:  6.1min finished


0:	learn: 0.1795776	total: 152ms	remaining: 15s
1:	learn: 0.1790527	total: 176ms	remaining: 8.62s
2:	learn: 0.1787317	total: 213ms	remaining: 6.88s
3:	learn: 0.1783762	total: 255ms	remaining: 6.11s
4:	learn: 0.1782251	total: 275ms	remaining: 5.23s
5:	learn: 0.1770979	total: 331ms	remaining: 5.19s
6:	learn: 0.1764945	total: 420ms	remaining: 5.58s
7:	learn: 0.1763279	total: 456ms	remaining: 5.25s
8:	learn: 0.1759226	total: 468ms	remaining: 4.73s
9:	learn: 0.1753987	total: 487ms	remaining: 4.38s
10:	learn: 0.1751098	total: 538ms	remaining: 4.35s
11:	learn: 0.1749980	total: 551ms	remaining: 4.04s
12:	learn: 0.1746652	total: 560ms	remaining: 3.75s
13:	learn: 0.1742475	total: 569ms	remaining: 3.49s
14:	learn: 0.1741523	total: 577ms	remaining: 3.27s
15:	learn: 0.1739709	total: 588ms	remaining: 3.09s
16:	learn: 0.1734987	total: 595ms	remaining: 2.91s
17:	learn: 0.1732263	total: 605ms	remaining: 2.75s
18:	learn: 0.1731609	total: 614ms	remaining: 2.62s
19:	learn: 0.1731012	total: 625ms	remaining

GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=<catboost.core.CatBoostRegressor object at 0x7f5257fb7be0>,
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'iterations': [100, 222], 'depth': [5, 7, 9], 'learning_rate': [0.01, 0.18831273426065617, 0.2], 'random_strength': [33], 'bagging_temperature': [0.01], 'od_type': ['Iter'], 'od_wait': [5, 10, 21]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [15]:
# 結果を表示
print(reg_cv.best_params_)
print(reg_cv.best_score_)

{'bagging_temperature': 0.01, 'depth': 5, 'iterations': 100, 'learning_rate': 0.18831273426065617, 'od_type': 'Iter', 'od_wait': 5, 'random_strength': 33}
0.08239181005757769


In [16]:
label = "label_low_20"

In [17]:
data_X = os.path.join(train_X_path, f"train_X_{label}.pkl")
with open(data_X , "rb") as f:
    train_X = pickle.load(f)
data_y = os.path.join(train_y_path, f"train_y_{label}.pkl")
with open(data_y , "rb") as f:
    train_y = pickle.load(f)

feature_columns = get_feature_columns(dfs, train_X, column_group='selected_low_columns')
# 訓練実施
reg_cv.fit(train_X[feature_columns].values, train_y.values)

Fitting 3 folds for each of 54 candidates, totalling 162 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


0:	learn: 0.0726521	total: 9.63ms	remaining: 953ms
1:	learn: 0.0725851	total: 16.5ms	remaining: 810ms
2:	learn: 0.0725356	total: 21.4ms	remaining: 691ms
3:	learn: 0.0725131	total: 25.7ms	remaining: 616ms
4:	learn: 0.0724594	total: 31.5ms	remaining: 598ms
5:	learn: 0.0724257	total: 37.5ms	remaining: 587ms
6:	learn: 0.0723455	total: 43.5ms	remaining: 578ms
7:	learn: 0.0722906	total: 48.8ms	remaining: 561ms
8:	learn: 0.0722606	total: 58.8ms	remaining: 594ms
9:	learn: 0.0722136	total: 65.2ms	remaining: 587ms
10:	learn: 0.0721287	total: 73.3ms	remaining: 593ms
11:	learn: 0.0721136	total: 81.8ms	remaining: 600ms
12:	learn: 0.0720380	total: 88.8ms	remaining: 594ms
13:	learn: 0.0719511	total: 97.5ms	remaining: 599ms
14:	learn: 0.0718905	total: 105ms	remaining: 597ms
15:	learn: 0.0718716	total: 117ms	remaining: 612ms
16:	learn: 0.0717834	total: 124ms	remaining: 604ms
17:	learn: 0.0717147	total: 132ms	remaining: 600ms
18:	learn: 0.0716400	total: 138ms	remaining: 587ms
19:	learn: 0.0715708	total:

75:	learn: 0.0713012	total: 494ms	remaining: 156ms
76:	learn: 0.0712685	total: 504ms	remaining: 151ms
77:	learn: 0.0712459	total: 509ms	remaining: 144ms
78:	learn: 0.0712029	total: 520ms	remaining: 138ms
79:	learn: 0.0711867	total: 524ms	remaining: 131ms
80:	learn: 0.0711408	total: 529ms	remaining: 124ms
81:	learn: 0.0711211	total: 536ms	remaining: 118ms
82:	learn: 0.0710905	total: 543ms	remaining: 111ms
83:	learn: 0.0710677	total: 548ms	remaining: 104ms
84:	learn: 0.0710423	total: 554ms	remaining: 97.7ms
85:	learn: 0.0710299	total: 560ms	remaining: 91.1ms
86:	learn: 0.0710054	total: 563ms	remaining: 84.2ms
87:	learn: 0.0709828	total: 567ms	remaining: 77.3ms
88:	learn: 0.0709647	total: 573ms	remaining: 70.9ms
89:	learn: 0.0709196	total: 580ms	remaining: 64.4ms
90:	learn: 0.0708886	total: 586ms	remaining: 58ms
91:	learn: 0.0708469	total: 590ms	remaining: 51.3ms
92:	learn: 0.0708178	total: 598ms	remaining: 45ms
93:	learn: 0.0707803	total: 604ms	remaining: 38.6ms
94:	learn: 0.0707609	tota

51:	learn: 0.0699259	total: 245ms	remaining: 226ms
52:	learn: 0.0698728	total: 251ms	remaining: 223ms
53:	learn: 0.0698317	total: 257ms	remaining: 219ms
54:	learn: 0.0698161	total: 261ms	remaining: 214ms
55:	learn: 0.0697942	total: 265ms	remaining: 208ms
56:	learn: 0.0697541	total: 269ms	remaining: 203ms
57:	learn: 0.0696945	total: 274ms	remaining: 198ms
58:	learn: 0.0696552	total: 278ms	remaining: 193ms
59:	learn: 0.0696404	total: 283ms	remaining: 188ms
60:	learn: 0.0696325	total: 287ms	remaining: 184ms
61:	learn: 0.0695777	total: 292ms	remaining: 179ms
62:	learn: 0.0695605	total: 297ms	remaining: 174ms
63:	learn: 0.0695177	total: 302ms	remaining: 170ms
64:	learn: 0.0694805	total: 306ms	remaining: 165ms
65:	learn: 0.0694283	total: 310ms	remaining: 160ms
66:	learn: 0.0693984	total: 316ms	remaining: 155ms
67:	learn: 0.0693714	total: 320ms	remaining: 151ms
68:	learn: 0.0693416	total: 325ms	remaining: 146ms
69:	learn: 0.0692976	total: 329ms	remaining: 141ms
70:	learn: 0.0692676	total: 333

...

187:	learn: 0.0533216	total: 2.6s	remaining: 471ms
188:	learn: 0.0532710	total: 2.61s	remaining: 456ms
189:	learn: 0.0531706	total: 2.62s	remaining: 442ms
190:	learn: 0.0530762	total: 2.63s	remaining: 428ms
191:	learn: 0.0529982	total: 2.64s	remaining: 413ms
192:	learn: 0.0529100	total: 2.66s	remaining: 399ms
193:	learn: 0.0528368	total: 2.67s	remaining: 386ms
194:	learn: 0.0526888	total: 2.68s	remaining: 371ms
195:	learn: 0.0526083	total: 2.69s	remaining: 357ms
196:	learn: 0.0524737	total: 2.71s	remaining: 344ms
197:	learn: 0.0524472	total: 2.72s	remaining: 329ms
198:	learn: 0.0523492	total: 2.73s	remaining: 316ms
199:	learn: 0.0522942	total: 2.75s	remaining: 302ms
200:	learn: 0.0522433	total: 2.75s	remaining: 288ms
201:	learn: 0.0521654	total: 2.77s	remaining: 274ms
202:	learn: 0.0520969	total: 2.78s	remaining: 260ms
203:	learn: 0.0519718	total: 2.79s	remaining: 246ms
204:	learn: 0.0518846	total: 2.8s	remaining: 233ms
205:	learn: 0.0517986	total: 2.82s	remaining: 219ms
206:	learn: 0.

[Parallel(n_jobs=1)]: Done 162 out of 162 | elapsed:  7.0min finished


0:	learn: 0.0745379	total: 14.5ms	remaining: 3.21s
1:	learn: 0.0737489	total: 35ms	remaining: 3.85s
2:	learn: 0.0731946	total: 46.6ms	remaining: 3.4s
3:	learn: 0.0728490	total: 70.7ms	remaining: 3.85s
4:	learn: 0.0724044	total: 78.6ms	remaining: 3.41s
5:	learn: 0.0719465	total: 85ms	remaining: 3.06s
6:	learn: 0.0712679	total: 91.6ms	remaining: 2.81s
7:	learn: 0.0710744	total: 97.9ms	remaining: 2.62s
8:	learn: 0.0708037	total: 105ms	remaining: 2.48s
9:	learn: 0.0704727	total: 113ms	remaining: 2.4s
10:	learn: 0.0701612	total: 119ms	remaining: 2.27s
11:	learn: 0.0700353	total: 125ms	remaining: 2.19s
12:	learn: 0.0698485	total: 131ms	remaining: 2.1s
13:	learn: 0.0696518	total: 138ms	remaining: 2.06s
14:	learn: 0.0694904	total: 147ms	remaining: 2.03s
15:	learn: 0.0693808	total: 159ms	remaining: 2.05s
16:	learn: 0.0692528	total: 169ms	remaining: 2.03s
17:	learn: 0.0691654	total: 182ms	remaining: 2.06s
18:	learn: 0.0690515	total: 195ms	remaining: 2.09s
19:	learn: 0.0689349	total: 203ms	remain

169:	learn: 0.0627592	total: 1.53s	remaining: 469ms
170:	learn: 0.0627166	total: 1.54s	remaining: 460ms
171:	learn: 0.0626827	total: 1.55s	remaining: 451ms
172:	learn: 0.0626676	total: 1.56s	remaining: 441ms
173:	learn: 0.0626519	total: 1.58s	remaining: 436ms
174:	learn: 0.0626153	total: 1.59s	remaining: 427ms
175:	learn: 0.0625822	total: 1.59s	remaining: 417ms
176:	learn: 0.0625608	total: 1.61s	remaining: 408ms
177:	learn: 0.0625323	total: 1.62s	remaining: 401ms
178:	learn: 0.0625086	total: 1.64s	remaining: 394ms
179:	learn: 0.0624870	total: 1.65s	remaining: 384ms
180:	learn: 0.0624562	total: 1.66s	remaining: 377ms
181:	learn: 0.0624312	total: 1.67s	remaining: 367ms
182:	learn: 0.0624049	total: 1.68s	remaining: 357ms
183:	learn: 0.0623912	total: 1.68s	remaining: 348ms
184:	learn: 0.0623706	total: 1.69s	remaining: 339ms
185:	learn: 0.0623436	total: 1.7s	remaining: 330ms
186:	learn: 0.0623123	total: 1.73s	remaining: 323ms
187:	learn: 0.0622848	total: 1.73s	remaining: 314ms
188:	learn: 0

GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=<catboost.core.CatBoostRegressor object at 0x7f5257fb7be0>,
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'iterations': [100, 222], 'depth': [5, 7, 9], 'learning_rate': [0.01, 0.18831273426065617, 0.2], 'random_strength': [33], 'bagging_temperature': [0.01], 'od_type': ['Iter'], 'od_wait': [5, 10, 21]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [18]:
# 結果を表示
print(reg_cv.best_params_)
print(reg_cv.best_score_)

{'bagging_temperature': 0.01, 'depth': 5, 'iterations': 222, 'learning_rate': 0.2, 'od_type': 'Iter', 'od_wait': 5, 'random_strength': 33}
0.21760066671896067


In [14]:
for label in tqdm(labels):
    data_X = os.path.join(train_X_path, f"train_X_{label}.pkl")
    with open(data_X , "rb") as f:
        train_X = pickle.load(f)
    data_y = os.path.join(train_y_path, f"train_y_{label}.pkl")
    with open(data_y , "rb") as f:
        train_y = pickle.load(f)
        
    feature_columns = get_feature_columns(dfs, train_X, column_group='fundamental+technical')
    # 訓練実施
    reg_cv.fit(train_X[feature_columns].values, train_y.values)

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


0:	learn: 0.1599492	total: 113ms	remaining: 24.9s
1:	learn: 0.1592691	total: 140ms	remaining: 15.4s
2:	learn: 0.1584998	total: 166ms	remaining: 12.1s
3:	learn: 0.1575706	total: 192ms	remaining: 10.5s
4:	learn: 0.1571170	total: 218ms	remaining: 9.46s
5:	learn: 0.1560890	total: 243ms	remaining: 8.73s
6:	learn: 0.1548959	total: 270ms	remaining: 8.3s
7:	learn: 0.1545646	total: 299ms	remaining: 8.01s
8:	learn: 0.1541441	total: 340ms	remaining: 8.04s
9:	learn: 0.1534789	total: 366ms	remaining: 7.75s
10:	learn: 0.1532661	total: 394ms	remaining: 7.56s
11:	learn: 0.1529457	total: 419ms	remaining: 7.34s
12:	learn: 0.1526601	total: 446ms	remaining: 7.18s
13:	learn: 0.1525200	total: 474ms	remaining: 7.05s
14:	learn: 0.1519625	total: 501ms	remaining: 6.92s
15:	learn: 0.1514368	total: 528ms	remaining: 6.8s
16:	learn: 0.1513033	total: 552ms	remaining: 6.66s
17:	learn: 0.1511497	total: 584ms	remaining: 6.62s
18:	learn: 0.1507912	total: 611ms	remaining: 6.53s
19:	learn: 0.1506301	total: 636ms	remaining

164:	learn: 0.1012757	total: 4.27s	remaining: 1.48s
165:	learn: 0.1010187	total: 4.29s	remaining: 1.45s
166:	learn: 0.1007683	total: 4.32s	remaining: 1.42s
167:	learn: 0.1005756	total: 4.34s	remaining: 1.4s
168:	learn: 0.1002738	total: 4.36s	remaining: 1.37s
169:	learn: 0.1001125	total: 4.38s	remaining: 1.34s
170:	learn: 0.0998389	total: 4.41s	remaining: 1.31s
171:	learn: 0.0995705	total: 4.43s	remaining: 1.29s
172:	learn: 0.0992575	total: 4.45s	remaining: 1.26s
173:	learn: 0.0988312	total: 4.48s	remaining: 1.24s
174:	learn: 0.0985210	total: 4.5s	remaining: 1.21s
175:	learn: 0.0982179	total: 4.53s	remaining: 1.18s
176:	learn: 0.0980136	total: 4.55s	remaining: 1.16s
177:	learn: 0.0978817	total: 4.58s	remaining: 1.13s
178:	learn: 0.0974666	total: 4.6s	remaining: 1.1s
179:	learn: 0.0972661	total: 4.62s	remaining: 1.08s
180:	learn: 0.0969581	total: 4.65s	remaining: 1.05s
181:	learn: 0.0967952	total: 4.67s	remaining: 1.02s
182:	learn: 0.0964346	total: 4.69s	remaining: 1s
183:	learn: 0.09611

103:	learn: 0.1349514	total: 2.45s	remaining: 2.77s
104:	learn: 0.1342016	total: 2.47s	remaining: 2.75s
105:	learn: 0.1339169	total: 2.5s	remaining: 2.73s
106:	learn: 0.1334818	total: 2.52s	remaining: 2.71s
107:	learn: 0.1330659	total: 2.55s	remaining: 2.69s
108:	learn: 0.1325110	total: 2.58s	remaining: 2.67s
109:	learn: 0.1317955	total: 2.61s	remaining: 2.66s
110:	learn: 0.1313788	total: 2.64s	remaining: 2.64s
111:	learn: 0.1309829	total: 2.67s	remaining: 2.62s
112:	learn: 0.1304687	total: 2.69s	remaining: 2.6s
113:	learn: 0.1298108	total: 2.71s	remaining: 2.57s
114:	learn: 0.1294132	total: 2.74s	remaining: 2.55s
115:	learn: 0.1290041	total: 2.76s	remaining: 2.52s
116:	learn: 0.1286349	total: 2.78s	remaining: 2.5s
117:	learn: 0.1284152	total: 2.81s	remaining: 2.47s
118:	learn: 0.1282067	total: 2.83s	remaining: 2.45s
119:	learn: 0.1280042	total: 2.86s	remaining: 2.43s
120:	learn: 0.1278155	total: 2.88s	remaining: 2.4s
121:	learn: 0.1275722	total: 2.9s	remaining: 2.38s
122:	learn: 0.127

...

42:	learn: 0.0685209	total: 1.03s	remaining: 4.3s
43:	learn: 0.0684091	total: 1.06s	remaining: 4.27s
44:	learn: 0.0682855	total: 1.08s	remaining: 4.24s
45:	learn: 0.0681105	total: 1.1s	remaining: 4.21s
46:	learn: 0.0680561	total: 1.12s	remaining: 4.18s
47:	learn: 0.0679079	total: 1.15s	remaining: 4.15s
48:	learn: 0.0678399	total: 1.17s	remaining: 4.12s
49:	learn: 0.0677428	total: 1.19s	remaining: 4.11s
50:	learn: 0.0675860	total: 1.22s	remaining: 4.09s
51:	learn: 0.0674523	total: 1.24s	remaining: 4.07s
52:	learn: 0.0672812	total: 1.27s	remaining: 4.04s
53:	learn: 0.0671737	total: 1.29s	remaining: 4.02s
54:	learn: 0.0669566	total: 1.31s	remaining: 3.99s
55:	learn: 0.0668699	total: 1.33s	remaining: 3.96s
56:	learn: 0.0667596	total: 1.36s	remaining: 3.93s
57:	learn: 0.0666633	total: 1.38s	remaining: 3.9s
58:	learn: 0.0665675	total: 1.4s	remaining: 3.88s
59:	learn: 0.0664596	total: 1.43s	remaining: 3.86s
60:	learn: 0.0663841	total: 1.45s	remaining: 3.83s
61:	learn: 0.0661840	total: 1.47s	r

202:	learn: 0.0474820	total: 4.87s	remaining: 456ms
203:	learn: 0.0473814	total: 4.9s	remaining: 432ms
204:	learn: 0.0472945	total: 4.93s	remaining: 409ms
205:	learn: 0.0472248	total: 4.95s	remaining: 385ms
206:	learn: 0.0471514	total: 4.98s	remaining: 361ms
207:	learn: 0.0470528	total: 5s	remaining: 337ms
208:	learn: 0.0469798	total: 5.03s	remaining: 313ms
209:	learn: 0.0468892	total: 5.05s	remaining: 288ms
210:	learn: 0.0468083	total: 5.07s	remaining: 264ms
211:	learn: 0.0467162	total: 5.1s	remaining: 240ms
212:	learn: 0.0466171	total: 5.12s	remaining: 216ms
213:	learn: 0.0465442	total: 5.14s	remaining: 192ms
214:	learn: 0.0464597	total: 5.17s	remaining: 168ms
215:	learn: 0.0463763	total: 5.19s	remaining: 144ms
216:	learn: 0.0462922	total: 5.21s	remaining: 120ms
217:	learn: 0.0461695	total: 5.23s	remaining: 96ms
218:	learn: 0.0460700	total: 5.26s	remaining: 72ms
219:	learn: 0.0459130	total: 5.29s	remaining: 48ms
220:	learn: 0.0458248	total: 5.31s	remaining: 24ms
221:	learn: 0.0457403

[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:   20.9s finished


0:	learn: 0.0745855	total: 26.3ms	remaining: 5.81s
1:	learn: 0.0737869	total: 53.8ms	remaining: 5.92s
2:	learn: 0.0726931	total: 79.9ms	remaining: 5.83s
3:	learn: 0.0719675	total: 107ms	remaining: 5.84s
4:	learn: 0.0715352	total: 134ms	remaining: 5.83s
5:	learn: 0.0709408	total: 164ms	remaining: 5.9s
6:	learn: 0.0701650	total: 191ms	remaining: 5.86s
7:	learn: 0.0700019	total: 216ms	remaining: 5.78s
8:	learn: 0.0696849	total: 246ms	remaining: 5.82s
9:	learn: 0.0692217	total: 269ms	remaining: 5.71s
10:	learn: 0.0691179	total: 295ms	remaining: 5.65s
11:	learn: 0.0688891	total: 318ms	remaining: 5.56s
12:	learn: 0.0686722	total: 340ms	remaining: 5.47s
13:	learn: 0.0685426	total: 364ms	remaining: 5.4s
14:	learn: 0.0683412	total: 387ms	remaining: 5.34s
15:	learn: 0.0681261	total: 412ms	remaining: 5.3s
16:	learn: 0.0680161	total: 437ms	remaining: 5.27s
17:	learn: 0.0679222	total: 461ms	remaining: 5.22s
18:	learn: 0.0676946	total: 485ms	remaining: 5.18s
19:	learn: 0.0675730	total: 508ms	remaini

167:	learn: 0.0513863	total: 4.13s	remaining: 1.33s
168:	learn: 0.0513429	total: 4.16s	remaining: 1.3s
169:	learn: 0.0512405	total: 4.19s	remaining: 1.28s
170:	learn: 0.0511768	total: 4.21s	remaining: 1.26s
171:	learn: 0.0511116	total: 4.24s	remaining: 1.23s
172:	learn: 0.0510634	total: 4.26s	remaining: 1.21s
173:	learn: 0.0509744	total: 4.29s	remaining: 1.18s
174:	learn: 0.0508799	total: 4.31s	remaining: 1.16s
175:	learn: 0.0507829	total: 4.34s	remaining: 1.13s
176:	learn: 0.0506718	total: 4.37s	remaining: 1.11s
177:	learn: 0.0506306	total: 4.39s	remaining: 1.08s
178:	learn: 0.0505023	total: 4.42s	remaining: 1.06s
179:	learn: 0.0504325	total: 4.44s	remaining: 1.04s
180:	learn: 0.0503590	total: 4.47s	remaining: 1.01s
181:	learn: 0.0503080	total: 4.49s	remaining: 987ms
182:	learn: 0.0502623	total: 4.51s	remaining: 962ms
183:	learn: 0.0501903	total: 4.54s	remaining: 938ms
184:	learn: 0.0501263	total: 4.57s	remaining: 914ms
185:	learn: 0.0500301	total: 4.6s	remaining: 890ms
186:	learn: 0.

In [15]:
# 結果を表示
print(reg_cv.best_params_)
print(reg_cv.best_score_)

{'bagging_temperature': 0.06584346890760226, 'depth': 9, 'iterations': 222, 'learning_rate': 0.18831273426065617, 'od_type': 'Iter', 'od_wait': 21, 'random_strength': 33}
0.2037709004629042


In [37]:
best_model = reg_cv.best_estimator_

In [40]:
SELECT_FIN_DATA_COLUMNS = ['Result_FinancialStatement FiscalYear', 'Result_FinancialStatement NetSales',
       'Result_FinancialStatement OperatingIncome', 'Result_FinancialStatement OrdinaryIncome',
       'Result_FinancialStatement NetIncome', 'Result_FinancialStatement TotalAssets',
       'Result_FinancialStatement NetAssets', 'Result_FinancialStatement CashFlowsFromOperatingActivities',
       'Result_FinancialStatement CashFlowsFromFinancingActivities',
       'Result_FinancialStatement CashFlowsFromInvestingActivities', 'Forecast_FinancialStatement FiscalYear',
       'Forecast_FinancialStatement NetSales', 'Forecast_FinancialStatement OperatingIncome',
       'Forecast_FinancialStatement OrdinaryIncome', 'Forecast_FinancialStatement NetIncome',
       'Result_Dividend FiscalYear', 'Result_Dividend QuarterlyDividendPerShare',
       'Result_Dividend AnnualDividendPerShare', 'Forecast_Dividend FiscalYear',
       'Forecast_Dividend QuarterlyDividendPerShare', 'Forecast_Dividend AnnualDividendPerShare',
       'IssuedShareEquityQuote IssuedShare','Section/Products', '33 Sector(Code)', '17 Sector(Code)']

In [41]:
# 学習用データセット定義
# ファンダメンタル情報
#fundamental_cols = dfs["stock_fin"].select_dtypes("float64").columns
fundamental_cols = pd.Index(SELECT_FIN_DATA_COLUMNS)
fundamental_cols = fundamental_cols[fundamental_cols != "Result_Dividend DividendPayableDate"]
fundamental_cols = fundamental_cols[fundamental_cols != "Local Code"]
# 価格変化率
returns_cols = [x for x in train_X.columns if "return" in x]
# テクニカル
technical_cols = [x for x in train_X.columns if (x not in fundamental_cols) and (x != "code")]

In [42]:
columns = {
    "fundamental_only": fundamental_cols,
    "return_only": returns_cols,
    "technical_only": technical_cols,
    "fundamental+technical": list(fundamental_cols) + list(technical_cols),
}

In [43]:
# 結果保存用
all_results = dict()
all_results['XGB'] = dict()

In [44]:
for label in labels:
    data_X = os.path.join(test_X_path, f"test_X_{label}.pkl")
    with open(data_X , "rb") as f:
        test_X = pickle.load(f)
    data_y = os.path.join(test_y_path, f"test_y_{label}.pkl")
    with open(data_y , "rb") as f:
        test_y = pickle.load(f)
    # データセット毎に処理
    for col in columns.keys():
        result = dict()
        # 目的変数毎に処理
        for label in tqdm(labels):
            if len(test_X[columns[col]]) > 0:
                # モデル取得
                pred_model = best_model
                # 学習
                pred_model.fit(train_X[columns[col]].values, train_y)
                # 結果データ作成
                result[label] = test_X[["code"]].copy()
                result[label]["datetime"] = test_X[columns[col]].index
                # 予測
                result[label]["predict"] = pred_model.predict(test_X[columns[col]].values)
                result[label]["predict_dir"] = np.sign(result[label]["predict"])
                # 実際の結果
                result[label]["actual"] = test_y.values
                result[label]["actual_dir"] = np.sign(result[label]["actual"])
                result[label].dropna(inplace=True)

        all_results['XGB'][col] = result

0:	learn: 0.0751787	total: 31.3ms	remaining: 6.91s
1:	learn: 0.0744944	total: 42.6ms	remaining: 4.69s
2:	learn: 0.0739736	total: 51.9ms	remaining: 3.79s
3:	learn: 0.0736432	total: 60.3ms	remaining: 3.29s
4:	learn: 0.0734331	total: 69.6ms	remaining: 3.02s
5:	learn: 0.0733386	total: 77.4ms	remaining: 2.79s
6:	learn: 0.0731921	total: 84.9ms	remaining: 2.61s
7:	learn: 0.0730814	total: 94.1ms	remaining: 2.52s
8:	learn: 0.0729703	total: 102ms	remaining: 2.42s
9:	learn: 0.0729198	total: 110ms	remaining: 2.33s
10:	learn: 0.0727952	total: 117ms	remaining: 2.24s
11:	learn: 0.0727114	total: 125ms	remaining: 2.19s
12:	learn: 0.0726724	total: 132ms	remaining: 2.13s
13:	learn: 0.0725661	total: 144ms	remaining: 2.14s
14:	learn: 0.0724807	total: 151ms	remaining: 2.09s
15:	learn: 0.0724228	total: 159ms	remaining: 2.05s
16:	learn: 0.0723807	total: 167ms	remaining: 2.02s
17:	learn: 0.0723424	total: 175ms	remaining: 1.98s
18:	learn: 0.0723225	total: 184ms	remaining: 1.96s
19:	learn: 0.0722746	total: 193ms

177:	learn: 0.0682829	total: 1.41s	remaining: 348ms
178:	learn: 0.0682655	total: 1.42s	remaining: 340ms
179:	learn: 0.0682433	total: 1.42s	remaining: 332ms
180:	learn: 0.0682176	total: 1.43s	remaining: 324ms
181:	learn: 0.0682049	total: 1.44s	remaining: 316ms
182:	learn: 0.0681808	total: 1.45s	remaining: 308ms
183:	learn: 0.0681633	total: 1.45s	remaining: 300ms
184:	learn: 0.0681527	total: 1.46s	remaining: 292ms
185:	learn: 0.0681307	total: 1.47s	remaining: 284ms
186:	learn: 0.0681204	total: 1.48s	remaining: 276ms
187:	learn: 0.0681015	total: 1.48s	remaining: 268ms
188:	learn: 0.0680897	total: 1.49s	remaining: 261ms
189:	learn: 0.0680786	total: 1.51s	remaining: 254ms
190:	learn: 0.0680542	total: 1.52s	remaining: 246ms
191:	learn: 0.0680358	total: 1.52s	remaining: 238ms
192:	learn: 0.0680235	total: 1.57s	remaining: 235ms
193:	learn: 0.0679984	total: 1.58s	remaining: 228ms
194:	learn: 0.0679788	total: 1.59s	remaining: 221ms
195:	learn: 0.0679679	total: 1.61s	remaining: 213ms
196:	learn: 

124:	learn: 0.0693763	total: 862ms	remaining: 669ms
125:	learn: 0.0693537	total: 869ms	remaining: 662ms
126:	learn: 0.0693454	total: 874ms	remaining: 654ms
127:	learn: 0.0693164	total: 881ms	remaining: 647ms
128:	learn: 0.0692984	total: 886ms	remaining: 639ms
129:	learn: 0.0692650	total: 891ms	remaining: 631ms
130:	learn: 0.0692440	total: 896ms	remaining: 622ms
131:	learn: 0.0692099	total: 903ms	remaining: 615ms
132:	learn: 0.0691936	total: 907ms	remaining: 607ms
133:	learn: 0.0691685	total: 912ms	remaining: 599ms
134:	learn: 0.0691402	total: 918ms	remaining: 592ms
135:	learn: 0.0691257	total: 924ms	remaining: 584ms
136:	learn: 0.0690943	total: 930ms	remaining: 577ms
137:	learn: 0.0690693	total: 937ms	remaining: 570ms
138:	learn: 0.0690366	total: 945ms	remaining: 565ms
139:	learn: 0.0690088	total: 954ms	remaining: 559ms
140:	learn: 0.0689902	total: 962ms	remaining: 553ms
141:	learn: 0.0689485	total: 968ms	remaining: 545ms
142:	learn: 0.0689372	total: 974ms	remaining: 538ms
143:	learn: 

0:	learn: 0.0758134	total: 22.9ms	remaining: 5.05s
1:	learn: 0.0755982	total: 36.1ms	remaining: 3.97s
2:	learn: 0.0753250	total: 55.8ms	remaining: 4.07s
3:	learn: 0.0748447	total: 61.4ms	remaining: 3.34s
4:	learn: 0.0742655	total: 67.9ms	remaining: 2.94s
5:	learn: 0.0740301	total: 72.8ms	remaining: 2.62s
6:	learn: 0.0738791	total: 77.2ms	remaining: 2.37s
7:	learn: 0.0736815	total: 81.6ms	remaining: 2.18s
8:	learn: 0.0735484	total: 86.1ms	remaining: 2.04s
9:	learn: 0.0732065	total: 90.5ms	remaining: 1.92s
10:	learn: 0.0731388	total: 96.2ms	remaining: 1.84s
11:	learn: 0.0730255	total: 101ms	remaining: 1.77s
12:	learn: 0.0727808	total: 107ms	remaining: 1.72s
13:	learn: 0.0727233	total: 111ms	remaining: 1.64s
14:	learn: 0.0725691	total: 115ms	remaining: 1.59s
15:	learn: 0.0724575	total: 119ms	remaining: 1.53s
16:	learn: 0.0724176	total: 124ms	remaining: 1.5s
17:	learn: 0.0723911	total: 128ms	remaining: 1.45s
18:	learn: 0.0723386	total: 133ms	remaining: 1.42s
19:	learn: 0.0723106	total: 138

174:	learn: 0.0686498	total: 1.06s	remaining: 284ms
175:	learn: 0.0686413	total: 1.06s	remaining: 278ms
176:	learn: 0.0686304	total: 1.07s	remaining: 272ms
177:	learn: 0.0686221	total: 1.08s	remaining: 266ms
178:	learn: 0.0686124	total: 1.08s	remaining: 259ms
179:	learn: 0.0686010	total: 1.08s	remaining: 253ms
180:	learn: 0.0685878	total: 1.09s	remaining: 247ms
181:	learn: 0.0685817	total: 1.09s	remaining: 240ms
182:	learn: 0.0685719	total: 1.1s	remaining: 234ms
183:	learn: 0.0685660	total: 1.1s	remaining: 228ms
184:	learn: 0.0685584	total: 1.11s	remaining: 222ms
185:	learn: 0.0685491	total: 1.11s	remaining: 215ms
186:	learn: 0.0685398	total: 1.11s	remaining: 209ms
187:	learn: 0.0685309	total: 1.12s	remaining: 202ms
188:	learn: 0.0685216	total: 1.12s	remaining: 196ms
189:	learn: 0.0685162	total: 1.12s	remaining: 189ms
190:	learn: 0.0685094	total: 1.13s	remaining: 183ms
191:	learn: 0.0685021	total: 1.13s	remaining: 177ms
192:	learn: 0.0684933	total: 1.14s	remaining: 171ms
193:	learn: 0.

...

165:	learn: 0.0624419	total: 1.53s	remaining: 518ms
166:	learn: 0.0624104	total: 1.55s	remaining: 512ms
167:	learn: 0.0623779	total: 1.57s	remaining: 505ms
168:	learn: 0.0623509	total: 1.59s	remaining: 498ms
169:	learn: 0.0623276	total: 1.6s	remaining: 491ms
170:	learn: 0.0623035	total: 1.61s	remaining: 481ms
171:	learn: 0.0622657	total: 1.62s	remaining: 471ms
172:	learn: 0.0622160	total: 1.63s	remaining: 460ms
173:	learn: 0.0621825	total: 1.63s	remaining: 451ms
174:	learn: 0.0621379	total: 1.64s	remaining: 441ms
175:	learn: 0.0621108	total: 1.65s	remaining: 431ms
176:	learn: 0.0620918	total: 1.66s	remaining: 421ms
177:	learn: 0.0620651	total: 1.66s	remaining: 411ms
178:	learn: 0.0620409	total: 1.67s	remaining: 402ms
179:	learn: 0.0620145	total: 1.68s	remaining: 392ms
180:	learn: 0.0619813	total: 1.69s	remaining: 382ms
181:	learn: 0.0619534	total: 1.7s	remaining: 373ms
182:	learn: 0.0619171	total: 1.7s	remaining: 363ms
183:	learn: 0.0618918	total: 1.71s	remaining: 353ms
184:	learn: 0.0

109:	learn: 0.0645634	total: 939ms	remaining: 956ms
110:	learn: 0.0645184	total: 946ms	remaining: 946ms
111:	learn: 0.0644754	total: 953ms	remaining: 936ms
112:	learn: 0.0644204	total: 961ms	remaining: 927ms
113:	learn: 0.0643890	total: 969ms	remaining: 918ms
114:	learn: 0.0643088	total: 1s	remaining: 931ms
115:	learn: 0.0642712	total: 1.01s	remaining: 922ms
116:	learn: 0.0642354	total: 1.02s	remaining: 913ms
117:	learn: 0.0642030	total: 1.03s	remaining: 909ms
118:	learn: 0.0641483	total: 1.04s	remaining: 905ms
119:	learn: 0.0640957	total: 1.06s	remaining: 899ms
120:	learn: 0.0640561	total: 1.07s	remaining: 892ms
121:	learn: 0.0640225	total: 1.08s	remaining: 885ms
122:	learn: 0.0639767	total: 1.09s	remaining: 879ms
123:	learn: 0.0639441	total: 1.1s	remaining: 874ms
124:	learn: 0.0639087	total: 1.12s	remaining: 867ms
125:	learn: 0.0638512	total: 1.13s	remaining: 860ms
126:	learn: 0.0638191	total: 1.14s	remaining: 852ms
127:	learn: 0.0637918	total: 1.15s	remaining: 846ms
128:	learn: 0.06

0:	learn: 0.0744932	total: 7.91ms	remaining: 1.75s
1:	learn: 0.0743229	total: 16.5ms	remaining: 1.82s
2:	learn: 0.0732429	total: 24ms	remaining: 1.75s
3:	learn: 0.0727763	total: 32.3ms	remaining: 1.76s
4:	learn: 0.0722628	total: 40.7ms	remaining: 1.76s
5:	learn: 0.0720482	total: 50.4ms	remaining: 1.81s
6:	learn: 0.0715228	total: 57.8ms	remaining: 1.77s
7:	learn: 0.0712059	total: 66.8ms	remaining: 1.79s
8:	learn: 0.0710187	total: 75ms	remaining: 1.77s
9:	learn: 0.0708083	total: 82.1ms	remaining: 1.74s
10:	learn: 0.0706774	total: 89.5ms	remaining: 1.72s
11:	learn: 0.0705451	total: 96.6ms	remaining: 1.69s
12:	learn: 0.0703661	total: 106ms	remaining: 1.7s
13:	learn: 0.0702207	total: 113ms	remaining: 1.68s
14:	learn: 0.0701727	total: 121ms	remaining: 1.67s
15:	learn: 0.0701050	total: 129ms	remaining: 1.66s
16:	learn: 0.0700296	total: 137ms	remaining: 1.65s
17:	learn: 0.0699840	total: 145ms	remaining: 1.64s
18:	learn: 0.0699503	total: 153ms	remaining: 1.63s
19:	learn: 0.0696128	total: 161ms	

170:	learn: 0.0618534	total: 1.53s	remaining: 457ms
171:	learn: 0.0618308	total: 1.55s	remaining: 450ms
172:	learn: 0.0618147	total: 1.56s	remaining: 441ms
173:	learn: 0.0617774	total: 1.57s	remaining: 433ms
174:	learn: 0.0617457	total: 1.58s	remaining: 425ms
175:	learn: 0.0617098	total: 1.6s	remaining: 418ms
176:	learn: 0.0616844	total: 1.61s	remaining: 410ms
177:	learn: 0.0616390	total: 1.62s	remaining: 402ms
178:	learn: 0.0615934	total: 1.64s	remaining: 394ms
179:	learn: 0.0615545	total: 1.65s	remaining: 385ms
180:	learn: 0.0615080	total: 1.66s	remaining: 377ms
181:	learn: 0.0614760	total: 1.67s	remaining: 367ms
182:	learn: 0.0614673	total: 1.68s	remaining: 357ms
183:	learn: 0.0614286	total: 1.68s	remaining: 347ms
184:	learn: 0.0613776	total: 1.69s	remaining: 338ms
185:	learn: 0.0613462	total: 1.7s	remaining: 329ms
186:	learn: 0.0613144	total: 1.71s	remaining: 319ms
187:	learn: 0.0612705	total: 1.72s	remaining: 311ms
188:	learn: 0.0612288	total: 1.73s	remaining: 302ms
189:	learn: 0.

108:	learn: 0.0644168	total: 1.06s	remaining: 1.1s
109:	learn: 0.0643651	total: 1.07s	remaining: 1.09s
110:	learn: 0.0643399	total: 1.07s	remaining: 1.07s
111:	learn: 0.0643003	total: 1.08s	remaining: 1.06s
112:	learn: 0.0642756	total: 1.08s	remaining: 1.05s
113:	learn: 0.0642263	total: 1.09s	remaining: 1.03s
114:	learn: 0.0641963	total: 1.1s	remaining: 1.02s
115:	learn: 0.0641600	total: 1.1s	remaining: 1.01s
116:	learn: 0.0640917	total: 1.11s	remaining: 998ms
117:	learn: 0.0640366	total: 1.12s	remaining: 987ms
118:	learn: 0.0639864	total: 1.13s	remaining: 975ms
119:	learn: 0.0639029	total: 1.13s	remaining: 964ms
120:	learn: 0.0638740	total: 1.14s	remaining: 953ms
121:	learn: 0.0638104	total: 1.15s	remaining: 941ms
122:	learn: 0.0637612	total: 1.16s	remaining: 930ms
123:	learn: 0.0637249	total: 1.16s	remaining: 918ms
124:	learn: 0.0636783	total: 1.17s	remaining: 908ms
125:	learn: 0.0636328	total: 1.18s	remaining: 897ms
126:	learn: 0.0635938	total: 1.18s	remaining: 886ms
127:	learn: 0.0

In [52]:
all_results['XGB']['fundamental+technical']

{'label_low_20':             code   datetime   predict  predict_dir   actual  actual_dir
 datetime                                                               
 2019-02-08  1301 2019-02-08 -0.039719         -1.0  0.07854         1.0
 2019-05-13  1301 2019-05-13 -0.042036         -1.0  0.04379         1.0
 2019-08-02  1301 2019-08-02 -0.048186         -1.0  0.00498         1.0
 2019-11-05  1301 2019-11-05 -0.040497         -1.0  0.00841         1.0
 2020-02-07  1301 2020-02-07 -0.010321         -1.0  0.01212         1.0
 2020-05-12  1301 2020-05-12 -0.045156         -1.0  0.06815         1.0
 2020-08-07  1301 2020-08-07 -0.046547         -1.0  0.02430         1.0
 2020-11-06  1301 2020-11-06 -0.009315         -1.0  0.02804         1.0
 2019-02-05  1332 2019-02-05 -0.067910         -1.0  0.08726         1.0
 2019-05-14  1332 2019-05-14 -0.056182         -1.0  0.02496         1.0
 2019-08-05  1332 2019-08-05 -0.039251         -1.0  0.04815         1.0
 2019-11-05  1332 2019-11-05 -0.056

In [53]:
results = []
for model in all_results.keys():
    for col in all_results[model]:
        tmp = pd.concat(all_results[model][col])
        tmp["model"] = model
        tmp["feature"] = col
        results.append(tmp)
results = pd.concat(results)
results["label"] = [x[0] for x in results.index]
results.head(5)

code   datetime   predict  predict_dir   actual  actual_dir model           feature  \
              datetime                                                                                          
label_high_20 2019-02-08  1301 2019-02-08 -0.045421         -1.0  0.07854         1.0   XGB  fundamental_only   
              2019-05-13  1301 2019-05-13 -0.034189         -1.0  0.04379         1.0   XGB  fundamental_only   
              2019-08-02  1301 2019-08-02 -0.062788         -1.0  0.00498         1.0   XGB  fundamental_only   
              2019-11-05  1301 2019-11-05 -0.059143         -1.0  0.00841         1.0   XGB  fundamental_only   
              2020-02-07  1301 2020-02-07 -0.052041         -1.0  0.01212         1.0   XGB  fundamental_only   

                                  label  
              datetime                   
label_high_20 2019-02-08  label_high_20  
              2019-05-13  label_high_20  
              2019-08-02  label_high_20  
              2019-11-05  label_high_20  
              2020-02-07  label_high_20

In [54]:
# 結果保存用変数
all_metrics = []

# データセット毎に処理
for feature in columns:
    matrix = dict()
    # 目的変数毎に処理
    for label in labels:
        # 処理対象データに絞り込み
        tmp_df = results[(results["model"] == "XGB") & (results["label"] == label) & (results["feature"] == feature)]
        # RMSE
        rmse = np.sqrt(mean_squared_error(tmp_df["predict"], tmp_df["actual"]))
        # 精度
        accuracy = accuracy_score(tmp_df["predict_dir"], tmp_df["actual_dir"])
        # 相関係数
        corr = np.corrcoef(tmp_df["actual"], tmp_df["predict"])[0, 1]
        # 順位相関
        spearman_corr = spearmanr(tmp_df["actual"], tmp_df["predict"])[0]
        # 結果を保存
        matrix[label] = [rmse, accuracy, spearman_corr,corr, corr**2, feature, model, tmp_df.shape[0]]
    res = pd.DataFrame.from_dict(matrix).T
    res.columns = ["RMSE","accuracy","spearman_corr","corr","R^2 score","feature", "model", "# of samples"]
    all_metrics.append(res)
all_metrics = pd.concat(all_metrics)
all_metrics.reset_index()

,index,RMSE,accuracy,spearman_corr,corr,R^2 score,feature,model,# of samples
0,label_low_20,0.231001,0.154452,-0.124725,-0.174387,0.0304109,fundamental_only,XGB,32515
1,label_high_20,0.231001,0.154452,-0.124725,-0.174387,0.0304109,fundamental_only,XGB,32515
2,label_low_20,0.237053,0.154113,-0.25069,-0.236044,0.0557169,return_only,XGB,32515
3,label_high_20,0.237053,0.154113,-0.25069,-0.236044,0.0557169,return_only,XGB,32515
4,label_low_20,0.235782,0.15805,-0.207399,-0.208541,0.0434893,technical_only,XGB,32515
5,label_high_20,0.235782,0.15805,-0.207399,-0.208541,0.0434893,technical_only,XGB,32515
6,label_low_20,0.236609,0.170075,-0.188987,-0.191727,0.0367591,fundamental+technical,XGB,32515
7,label_high_20,0.236609,0.170075,-0.188987,-0.191727,0.0367591,fundamental+technical,XGB,32515


In [24]:
tmp = all_metrics.drop(columns=["# of samples"])

In [25]:
tmp.to_csv('result_XGB.csv', index=None)

In [26]:
tmp = pd.read_csv('result_XGB.csv')

In [27]:
tmp.groupby(['feature', 'model']).mean()

,,RMSE,accuracy,spearman_corr,corr,R^2 score
feature,model,,,,,
fundamental+technical,XGB,0.104509,0.864155,0.159079,0.153215,0.023475
fundamental_only,XGB,0.104447,0.866738,0.140670,0.143726,0.020657
return_only,XGB,0.107356,0.867784,0.080475,0.058545,0.003428
technical_only,XGB,0.105467,0.861449,0.153756,0.146159,0.021362
